<a href="https://colab.research.google.com/github/GTaumaturgo/Experimental/blob/master/Cadastrador_de_despesas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplificador de gastos em viagem

In [98]:
#@title Imports

# !pip install --upgrade gspread

from google.colab import auth
import gspread
from google.auth import default
from oauth2client.client import GoogleCredentials
import pandas as pd


In [99]:
#@title Util


def run_gspread_auth():
  """
  Authorizes with the Google Sheets API and returns the gspread client.

  Returns:
    The authorized gspread client object.
  """

  # Authenticate with Google Sheets API
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)

  return gc


def read_from_sheets(spreadsheet_id, sheet_name):
  """
  Reads data from a Google Spreadsheet and returns it as a pandas DataFrame.

  Args:
    spreadsheet_id: The ID of the Google Spreadsheet.
    sheet_name: The name of the sheet to read from.

  Returns:
    A pandas DataFrame containing the data from the spreadsheet.
  """
  import pandas as pd

  # Open the spreadsheet and sheet
  sh = run_gspread_auth().open_by_key(spreadsheet_id)
  worksheet = sh.worksheet(sheet_name)

  # Get all values from the sheet
  data = worksheet.get_all_values()

  # Create a pandas DataFrame from the data
  df = pd.DataFrame(data[1:], columns=data[0])  # Assuming first row is header

  return df

def write_to_sheets(df, spreadsheet_id, sheet_name):
  """
  Writes a pandas DataFrame to a Google Spreadsheet.

  Args:
    df: The pandas DataFrame to write.
    spreadsheet_id: The ID of the Google Spreadsheet.
    sheet_name: The name of the sheet to write to.
  """
  # Open the spreadsheet and sheet
  sh = run_gspread_auth().open_by_key(spreadsheet_id)
  worksheet = sh.worksheet(sheet_name)

  # Clear the existing data in the sheet
  worksheet.clear()

  # Write the DataFrame to the sheet
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [100]:
# @title Parameters
LIST_OF_PEOPLE = ['tartarugo', 'tiovski', 'andrey'] # @param
SHEET_ID = '1vma-aqzudwA14Coj4lfX3FNeyoVILX4erRCQN7Vd9YI' #@param {type: "string"}
# title of the sheet (aka tab)
TAB_NAME = 'data' #@param {type: "string"}
NUM_PEOPLE = len(LIST_OF_PEOPLE)
# if false, we'll read the saved sheet instead of an empty one
CREATE_NEW_SHEET = True #@param {type: "boolean"}

## Create new spreadsheet

In [101]:
if CREATE_NEW_SHEET:
  variable_columns = []
  for i in range(NUM_PEOPLE):
    variable_columns.append('pct_owes_%d' % i)
  for i in range(NUM_PEOPLE):
    variable_columns.append( 'pct_paid_%d' % i)

  columns = ['what', 'price_usd', 'date'] + variable_columns
  expenses_df = pd.DataFrame(columns=columns)
  expenses_df

Now use the output DF to create N variables manually. (interactive form)

In [102]:
if CREATE_NEW_SHEET:
  for col in expenses_df.columns:
    print(col)

what
price_usd
date
pct_owes_0
pct_owes_1
pct_owes_2
pct_paid_0
pct_paid_1
pct_paid_2


## Reload existing sheet

In [103]:
if not CREATE_NEW_SHEET:
  expenses_df = read_from_sheets(SHEET_ID, TAB_NAME)

In [104]:
#@title Validate expense entry

def is_valid_expense(new_expense):
  if new_expense['what'] == '':
    return False
  if new_expense['price_usd'] <= 0:
    return False

  total_pct_paid = 0
  total_pct_owed = 0
  for i in range(NUM_PEOPLE):
    total_pct_paid += new_expense['pct_paid_%d' % i]
    total_pct_owed += new_expense['pct_owes_%d' % i]
  if total_pct_paid != 100 or total_pct_owed != 100:
    return False
  return True


In [105]:

WHAT = 'Steam deck' #@param {type: "string"}
PRICE_USD = 1 #@param {type: "number"}
PCT_OWES_0 = 100 #@param {type: "number"}
PCT_OWES_1 = 0 #@param {type: "number"}
PCT_OWES_2 = 0 #@param {type: "number"}
PCT_PAID_0 = 0 #@param {type: "number"}
PCT_PAID_1 = 100 #@param {type: "number"}
PCT_PAID_2 = 0 #@param {type: "number"}
DATE = '2023-04-01' #@param {type: "date"}

new_expense = {
    'what': WHAT,
    'price_usd': PRICE_USD,
    'pct_owes_0': PCT_OWES_0,
    'pct_owes_1': PCT_OWES_1,
    'pct_owes_2': PCT_OWES_2,
    'pct_paid_0': PCT_PAID_0,
    'pct_paid_1': PCT_PAID_1,
    'pct_paid_2': PCT_PAID_2,
    'date': DATE
}

if is_valid_expense(new_expense):
  print('Valid expense! Success.')
  expenses_df.loc[len(expenses_df)] = new_expense
else:
  print('Invalid expense! aborted')


Valid expense! Success.


##Check expenses

In [106]:
expenses_df

,what,price_usd,date,pct_owes_0,pct_owes_1,pct_owes_2,pct_paid_0,pct_paid_1,pct_paid_2
0,Steam deck,1,2023-04-01,100,0,0,0,100,0


In [107]:
# @title Delete entries

INDEX_TO_DELETE = -1 #@param {type: "number"}
if INDEX_TO_DELETE >= 0:
  expenses_df = expenses_df.drop(INDEX_TO_DELETE)
  print('Result:')
  expenses_df

In [108]:
def is_valid_expense(new_expense):
  if new_expense['what'] == '':
    return False
  if new_expense['price_usd'] <= 0:
    return False

  total_pct_paid = 0
  total_pct_owed = 0
  for i in range(NUM_PEOPLE):
    total_pct_paid += new_expense['pct_paid_%d' % i]
    total_pct_owed += new_expense['pct_owes_%d' % i]
  if total_pct_paid != 100 or total_pct_owed != 100:
    return False
  return True


In [109]:
write_to_sheets(expenses_df, SHEET_ID, TAB_NAME)